# 导出带注释的po文件

In [ ]:
# 选择角色
char = "wilson"
char = char.upper()
# 环境
dst_base = "e:/dst/scripts"
speech_base = f"{dst_base}/root/"
comment_file = "E:/dst/other/comment202405_other.json"
# 另一个角色
char_candidates = ["willow", "wendy"]
char_candidate = char_candidates[0]
if char == char_candidate:
    char_candidate = char_candidates[1]

In [ ]:
# 读取speech_char.lua
import os
import json
from lupa.lua51 import LuaRuntime
import lupa.lua51 as lupa


def read_lua_file(file_path):
    """
    Reads a Lua file and returns the value returned by the Lua file.

    Args:
        file_path (str): The path to the Lua file.

    Returns:
        dict: The value returned by the Lua file.
    """
    try:
        # Create a Lua runtime
        lua = LuaRuntime(unpack_returned_tuples=True)

        # Load the Lua file
        with open(file_path, "r") as file:
            lua_code = file.read()
        lua_chunk = lua.execute(lua_code)

        # Call the Lua chunk to get the returned value
        return_value = lua_chunk
        return return_value
    except Exception as e:
        print(f"Error reading Lua file: {e}")
        return None


def lua_table_to_python(table, parent_key=None):
    """
    Recursively traverses a Lua table and builds a tree-like Python dictionary.

    Args:
        table (lupa.lua.LuaTable): The Lua table to be traversed.
        parent_key (str, optional): The key of the parent table, if any.

    Returns:
        dict: The tree-like Python dictionary.
    """
    python_dict = {}

    for key, value in table.items():
        if lupa.lua_type(value) == "table":
            # Recursive case: the value is another Lua table
            python_dict[str(key)] = lua_table_to_python(value, str(key))
        else:
            # Base case: the value is a leaf node
            python_dict[str(key)] = str(value)

    return python_dict



In [ ]:
#首先读取speech_char.lua
speech_char=read_lua_file(f"{speech_base}/speech_{char}.lua")
speech_char=lua_table_to_python(speech_char)
#其次读取对照speech_candidate.lua
speech_candidate=read_lua_file(f"{speech_base}/speech_{char_candidate}.lua")
speech_candidate=lua_table_to_python(speech_candidate)

In [ ]:
# 展平字典
def flatten_dict(d, parent_key="", sep="."):
    """
    Flattens a nested dictionary into a flat dictionary with concatenated keys.

    Args:
        d (dict): The input nested dictionary to be flattened.
        parent_key (str): The current concatenated key (if any).
        sep (str): The separator character used to concatenate the keys.

    Returns:
        dict: The flattened dictionary with concatenated keys.
    """
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_dict(v, new_key, sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

In [ ]:
#转换为平坦字典
string_char = flatten_dict(speech_char)
string_candidate = flatten_dict(speech_candidate)

In [ ]:
# 取交集
def dict_intersect(dict1, dict2):
    """
    Returns the intersection of two dictionaries, excluding any key-value pairs where the value contains the string "only_used".

    Args:
        dict1 (dict): The first dictionary.
        dict2 (dict): The second dictionary.

    Returns:
        dict: The intersection of the two dictionaries, excluding any key-value pairs where the value contains "only_used".
    """
    result = {}
    for key in set(dict1.keys()) & set(dict2.keys()):
        if "only_used" not in str(dict1[key]) and "only_used" not in str(dict2[key]):
            result[key] = dict1[key]
    return result

In [ ]:
# 求出交集
intersect_char = dict_intersect(string_char, string_candidate)
intersect_key = set(intersect_char.keys())

In [ ]:
import json

__cached_path = ""
__cached_json = None


def loadjson(path):
    global __cached_path, __cached_json
    if path == __cached_path:
        return __cached_json
    with open(path, "r", encoding="utf-8") as f:
        __cached_json = json.load(f)
        __cached_path = path
        return __cached_json

In [ ]:
# 加载make po文件
from make_po import PO
import polib

# 打开标准po文件
chinese_po = PO(f"{dst_base}/languages/chinese_s.po")
# 打开注释文件
comment_dict = loadjson(comment_file)

In [ ]:
# 查找注释
def find_comment(id):
    if id in comment_dict:
        return comment_dict[id]
    if id.startswith("DESCRIBE"):
        item = id.split(".")[1]
        item_key = f"STRINGS.NAMES.{item}"
        item_generic_key = f"STRINGS.NAMES.{item}.GENERIC"
        ret = []
        if item_key in chinese_po.po_dict:
            ret.append(f"物品名:{chinese_po.po_dict[item_key]}")
        elif item_generic_key in chinese_po.po_dict:
            print("Found a generic item", item)
            ret.append(f"物品名:{chinese_po.po_dict[item_generic_key]}")
        item_recipe_key = f"STRINGS.RECIPE_DESC.{item}"
        if item_recipe_key in chinese_po.po_dict:
            ret.append(f"配方描述:{chinese_po.po_dict[item_recipe_key]}")
        return " ".join(ret)
    return ""

In [ ]:
# 提取全部key
all_keys = intersect_key
prefix = "STRINGS.CHARACTERS."
char_alias = char if char != "WILSON" else "GENERIC"
prefix_char = f"{prefix}{char_alias}."
prefix_pure = f"{prefix}{char}."
extract_po = PO()
extract_po.po = extract_po.po or polib.POFile()
for i in chinese_po.po:
    id = i.msgctxt
    en = i.msgid
    zh = i.msgstr
    cmt = None
    pure_id = id
    if pure_id.startswith(prefix_char):
        pure_id = pure_id[len(prefix_char) :]
        if pure_id in all_keys:
            cmt = find_comment(pure_id)
            pure_id = prefix_pure + pure_id
            extract_po.add(
                polib.POEntry(msgctxt=pure_id, msgid=en, msgstr=zh, comment=cmt)
            )

In [ ]:
extract_po.po_path=f"{char}.po"
extract_po.save(f"{char}.po")